Load data

In [1]:
from experiment.src.data_loader import read_detected_data, read_metadata, join_label, get_missing, eval_no_model, \
    get_y_labels, eval_with_model
from copy import deepcopy
from experiment.src.split import load_fixed_split
from experiment.src.features import prepare_data
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Input, Concatenate
from tensorflow.keras.models import Model
from skopt.space import Integer, Categorical
import matplotlib.pyplot as plt
import gc
from datetime import datetime
from skopt.utils import use_named_args
from keras.callbacks import EarlyStopping 

from experiment.src.prepare_data import prepare_train_data
import numpy as np
import yaml
from pathlib import Path
import json 
from typing import Tuple, List


2024-02-22 09:24:58.716420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 09:24:58.716621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 09:24:58.719146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 09:24:58.730594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 09:25:03.086614: W tensorflow/compiler/tf2

In [2]:
def get_predictions_keras(model: Model, data: List[np.ndarray]) -> Tuple[np.ndarray, np.ndarray]:
    probability = model.predict(data).ravel()
    prediction = probability > 0.5

    return prediction, probability

In [3]:
def get_class_weights(y_data):
        class_weights = compute_class_weight('balanced', classes=np.unique(y_data), y=y_data)
        return dict(enumerate(class_weights))

In [4]:
def save_history_acc_loss(history, dst_dir):
    # summarize history for accuracy
    plt.clf()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(dst_dir + '/accuracy.png')
    
    plt.clf()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(dst_dir + '/loss.png')
    plt.clf()

In [5]:
def save_model(model, params, train_history, eval_res):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_file_dir = f"results/ml_{eval_res['F1']:.6f}-{current_time}"
    Path(model_file_dir).mkdir(parents=True, exist_ok=True)
    model.save(f"{model_file_dir}/model")
    save_history_acc_loss(train_history, f"{model_file_dir}/")
    with open(f"{model_file_dir}/info.txt", 'w') as f:
        f.write(json.dumps(params, default=str))
        f.write('\n')
        f.write(json.dumps(eval_res, default=str))
    print(f'Saving model {model_file_dir}')

In [6]:
tf.config.list_physical_devices('GPU')

2024-02-22 09:25:08.457688: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 09:25:08.540434: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 09:25:08.540823: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
with open('env.yaml', 'r') as yaml_settings:
    args = yaml.load(yaml_settings, yaml.FullLoader)
cred_data_location = args[0]['cred_data_location']
print(cred_data_location)

/mnt/c/Users/yuliia.t/CredData


In [8]:
detected_data = read_detected_data("wsl/result.json")
meta_data = read_metadata(f"{cred_data_location}/meta")

detected_data_copy = deepcopy(detected_data)
meta_data_copy = deepcopy(meta_data)

    # Combine original and augmented data together
aug_detected_data = read_detected_data("wsl/result_aug_data.json", "aug_data/")
detected_data.update(aug_detected_data)
aug_metadata = read_metadata(f"{cred_data_location}/aug_data/meta", "aug_data/")
meta_data.update(aug_metadata)

Reading detections from wsl/result.json
Detected 19780 unique lines!
23805 detections in total
Reading meta from /mnt/c/Users/yuliia.t/CredData/meta
Loaded 64394 lines from meta!
64428 lines in meta in total
Reading detections from wsl/result_aug_data.json
Detected 144330 unique lines!
155787 detections in total
Reading meta from /mnt/c/Users/yuliia.t/CredData/aug_data/meta
Loaded 40331 lines from meta!
40331 lines in meta in total


In [9]:
df = join_label(detected_data, meta_data)

train_repo_list, test_repo_list = load_fixed_split()

df_train = df[df["repo"].isin(train_repo_list)]

In [10]:
print(f"Train size: {len(df_train)}")

df_train = df_train.drop_duplicates(subset=["line", "ext"])
print(f"Train size after drop_duplicates: {len(df_train)}")

X_train_value, X_train_features = prepare_data(df_train)
y_train = get_y_labels(df_train)

print(f"Class-1 prop on train: {np.mean(y_train):.2f}")

Train size: 16625
Train size after drop_duplicates: 11457
Class-1 prop on train: 0.22


In [11]:
get_class_weights(y_train)

{0: 0.641633064516129, 1: 2.2651245551601424}

In [12]:
vocab_size = X_train_value.shape[-1]
feature_size = X_train_features.shape[-1]

Setup parameters

In [13]:
DEFAULT_METRICS = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]# 'sparse_categorical_accuracy']

2024-02-22 09:27:09.626340: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 09:27:09.626597: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 09:27:09.626726: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 09:27:10.975102: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 09:27:10.975250: I external/local_xla/xla/stream_executor

In [14]:
lstm_units = 128
dense_1_units = 128
batch_size = 64
epochs = 100
activation = ['softmax','relu', 'tanh', 'sigmoid']#'softmax', 'linear']
# Hyperparameters
space = [
    # Model params
    Integer(1, 500, name="lstm_units"),
    Integer(1, 300, name="dense_1_units"),
    Categorical(activation, name="activation"),
    Categorical([0.1, 0.01, 0.001, 0.0001, 0.00001], name='learning_rate'),

    # Context params
    Integer(10, 80,  name="epochs"),
    Integer(1, 128, name="batch_size")]
    

In [15]:
best_loss = 1
f1 = 0

In [16]:
print("Validate results on the test subset")
df = join_label(detected_data_copy, meta_data_copy)
df_missing = get_missing(detected_data_copy, meta_data_copy)
df_test = df[df["repo"].isin(test_repo_list)]
df_test = df_test[df_test["value"].notna()]

df_missing_test = df_missing[df_missing["repo"].isin(test_repo_list)]
X_test_value, X_test_features = prepare_data(df_test)
y_test = get_y_labels(df_test)

print(f"Test size: {len(df_test)}")

Validate results on the test subset
Test size: 3155


In [17]:
early_stopping_monitor = EarlyStopping(patience=4)

@use_named_args(space)
def train_model(**params):
    print(params)
    lstm_input = Input(shape=(None, vocab_size), name="line_input")
    lstm_branch = Bidirectional(LSTM(params['lstm_units']))(lstm_input)
    
    feature_input = Input(shape=(feature_size, ), name="feature_input") 
    joined_features = Concatenate()([lstm_branch, feature_input])
    x = Dense(params['dense_1_units'], activation=params['activation'], name="Dense_1")(joined_features)
    x = Dense(1, activation='sigmoid', name="prediction")(x)
    
    model = Model([lstm_input, feature_input], x)
    
    adam_optimizer =  tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=DEFAULT_METRICS)
    hist = model.fit(
        [X_train_value, X_train_features],
        y_train,
        #validation_split=0.2,
        validation_data = ([X_test_value, X_test_features], y_test),
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        class_weight={
           0: 0.641633064516129, 1: 2.2651245551601424})
        #callbacks=[early_stopping_monitor])

    print(f"Min training loss={min(hist.history['loss'])}, min validation loss={min(hist.history['val_loss'])}")
    test_predictions, test_probabilities = get_predictions_keras(model, [X_test_value, X_test_features])
    print("Results on test with model:")
    val_res = eval_with_model(df_test, df_missing_test, test_predictions)
        
    save_model(model, params, hist, val_res)
    del model
    gc.collect()
    return val_res['F1'] 

In [ ]:
from skopt import gp_minimize
N_TRIALS = 40
SEED = 5       # 2020
np.int = int

search_result = gp_minimize(func=train_model, dimensions=space, n_calls=N_TRIALS,random_state=SEED)

{'lstm_units': 29, 'dense_1_units': 250, 'activation': 'sigmoid', 'learning_rate': 0.1, 'epochs': 16, 'batch_size': 51}


2024-02-22 09:27:30.223144: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/16


2024-02-22 09:27:40.756415: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-22 09:27:41.807348: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8740b2dd70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-22 09:27:41.807426: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2024-02-22 09:27:41.823168: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708586861.990588    3425 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


225/225 [==============================] - 40s 140ms/step - loss: 0.4184 - accuracy: 0.8537 - precision: 0.6237 - recall: 0.8501 - val_loss: 0.4110 - val_accuracy: 0.9014 - val_precision: 0.6560 - val_recall: 0.9224
Epoch 2/16
225/225 [==============================] - 32s 144ms/step - loss: 0.2356 - accuracy: 0.9155 - precision: 0.7468 - recall: 0.9340 - val_loss: 0.3277 - val_accuracy: 0.9201 - val_precision: 0.7008 - val_recall: 0.9513
Epoch 3/16
225/225 [==============================] - 29s 128ms/step - loss: 0.2627 - accuracy: 0.9063 - precision: 0.7358 - recall: 0.8976 - val_loss: 0.2501 - val_accuracy: 0.8935 - val_precision: 0.6485 - val_recall: 0.8592
Epoch 4/16
225/225 [==============================] - 43s 191ms/step - loss: 0.3180 - accuracy: 0.8935 - precision: 0.7041 - recall: 0.8928 - val_loss: 0.2766 - val_accuracy: 0.9087 - val_precision: 0.7367 - val_recall: 0.7473
Epoch 5/16
225/225 [==============================] - 37s 163ms/step - loss: 0.2798 - accuracy: 0.9048 

...

Saving model results/ml_0.440228-20240222_104009
{'lstm_units': 417, 'dense_1_units': 268, 'activation': 'tanh', 'learning_rate': 0.001, 'epochs': 16, 'batch_size': 127}
Epoch 1/16
91/91 [==============================] - 262s 3s/step - loss: 0.2931 - accuracy: 0.8758 - precision: 0.6685 - recall: 0.8073 - val_loss: 0.1177 - val_accuracy: 0.9632 - val_precision: 0.9345 - val_recall: 0.8502
Epoch 2/16
91/91 [==============================] - 283s 3s/step - loss: 0.1689 - accuracy: 0.9388 - precision: 0.8096 - recall: 0.9450 - val_loss: 0.1570 - val_accuracy: 0.9436 - val_precision: 0.7892 - val_recall: 0.9260
Epoch 3/16
91/91 [==============================] - 283s 3s/step - loss: 0.1308 - accuracy: 0.9536 - precision: 0.8520 - recall: 0.9557 - val_loss: 0.1850 - val_accuracy: 0.9271 - val_precision: 0.7225 - val_recall: 0.9495
Epoch 4/16
91/91 [==============================] - 283s 3s/step - loss: 0.1216 - accuracy: 0.9542 - precision: 0.8521 - recall: 0.9589 - val_loss: 0.1260 - val_

2024-02-22 11:56:25,428 | INFO | builder_impl.py:801 | Assets written to: results/ml_0.757943-20240222_115617/model/assets


Saving model results/ml_0.757943-20240222_115617
{'lstm_units': 239, 'dense_1_units': 206, 'activation': 'tanh', 'learning_rate': 0.001, 'epochs': 49, 'batch_size': 48}
Epoch 1/49
239/239 [==============================] - 170s 695ms/step - loss: 0.2423 - accuracy: 0.9017 - precision: 0.7399 - recall: 0.9114 - val_loss: 0.1990 - val_accuracy: 0.9151 - val_precision: 0.6829 - val_recall: 0.9639
Epoch 2/49
239/239 [==============================] - 162s 678ms/step - loss: 0.1418 - accuracy: 0.9487 - precision: 0.8366 - recall: 0.9537 - val_loss: 0.0981 - val_accuracy: 0.9712 - val_precision: 0.9097 - val_recall: 0.9278
Epoch 3/49
239/239 [==============================] - 164s 686ms/step - loss: 0.1220 - accuracy: 0.9548 - precision: 0.8529 - recall: 0.9609 - val_loss: 0.1302 - val_accuracy: 0.9534 - val_precision: 0.8165 - val_recall: 0.9477
Epoch 4/49
239/239 [==============================] - 166s 693ms/step - loss: 0.1154 - accuracy: 0.9592 - precision: 0.8682 - recall: 0.9612 - val_

2024-02-22 14:12:13,951 | INFO | builder_impl.py:801 | Assets written to: results/ml_0.787106-20240222_141206/model/assets


Saving model results/ml_0.787106-20240222_141206
{'lstm_units': 346, 'dense_1_units': 245, 'activation': 'tanh', 'learning_rate': 1e-05, 'epochs': 60, 'batch_size': 86}
Epoch 1/60
134/134 [==============================] - 270s 2s/step - loss: 0.6650 - accuracy: 0.4002 - precision: 0.2985 - recall: 0.9342 - val_loss: 0.5373 - val_accuracy: 0.8222 - val_precision: 0.4955 - val_recall: 0.6913
Epoch 2/60
134/134 [==============================] - 267s 2s/step - loss: 0.5643 - accuracy: 0.7445 - precision: 0.4435 - recall: 0.6180 - val_loss: 0.4736 - val_accuracy: 0.8403 - val_precision: 0.5354 - val_recall: 0.6823
Epoch 3/60
134/134 [==============================] - 266s 2s/step - loss: 0.5200 - accuracy: 0.7802 - precision: 0.5017 - recall: 0.6291 - val_loss: 0.4455 - val_accuracy: 0.8691 - val_precision: 0.6080 - val_recall: 0.7166
Epoch 4/60
134/134 [==============================] - 266s 2s/step - loss: 0.4792 - accuracy: 0.8309 - precision: 0.6080 - recall: 0.6592 - val_loss: 0.3818

In [ ]:
gc.collect()